In [1]:
import pandas as pd
import numpy as np
import math

## 门店基础信息

poi_id : 门店id

mt_poi_cate2_name : 二级品类（火锅，东北菜）

price_person : 人均价

has_parking_area : 停车位

has_booth : 是否有包厢

is_dining : 是否正轻餐

barea_id : 商圈

mt_score : 美团综合评分

dp_score : 点评综合评分

dp_evn_score : 环境

dp_taste_score : 口味

dp_service_score : 服务

dp_star : 星级

poi_zlf : 门店质量分

poi_rank : 门店排名 KA，A是头，B是腰，C，D是尾

In [20]:
df_poi_info = pd.read_csv('../raw_data/poi_info.txt', sep='\t')

In [24]:
df_poi_info.head(10)

,poi_id,mt_poi_cate2_name,price_person,has_parking_area,has_booth,is_dining,barea_id,mt_score,dp_score,dp_evn_score,dp_taste_score,dp_service_score,dp_star,poi_zlf,poi_rank
0,9456,70,108.0,0,NaN,0.0,6424,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9533,70,46.0,1,NaN,0.0,18729,41.0,NaN,NaN,NaN,NaN,NaN,26.743363,C
2,10737,70,72.0,0,NaN,0.0,5056,39.0,NaN,NaN,NaN,NaN,NaN,65.164556,B
3,11157,13,170.0,1,NaN,1.0,7160,50.0,NaN,NaN,NaN,NaN,NaN,37.265228,B
4,11304,27,128.0,1,NaN,1.0,1316,42.0,NaN,NaN,NaN,NaN,NaN,83.275626,A
5,11983,70,50.0,0,NaN,0.0,18503,36.0,NaN,NaN,NaN,NaN,NaN,50.758987,B
6,12585,70,68.0,0,NaN,0.0,237,39.0,NaN,NaN,NaN,NaN,NaN,36.906072,B
7,12711,33,106.0,0,NaN,1.0,7172,39.0,NaN,NaN,NaN,NaN,NaN,47.172788,B
8,13005,46,144.0,0,NaN,1.0,1927,42.0,NaN,NaN,NaN,NaN,NaN,89.163909,A
9,13012,27,176.0,1,NaN,1.0,5186,30.0,NaN,NaN,NaN,NaN,NaN,126.678827,KA


## 访问数据

poi_pv : 门店访问数

partition_date : 分区的日期

poi_id : 门店id

poi_uv : 用户访问数

deal_id : 项目id

In [25]:
df_poi_deal_pv_train = pd.read_csv('../raw_data/poi_deal_pv_train.txt', sep='\t')

In [26]:
df_poi_deal_pv_train.head(5)

,deal_id,poi_id,poi_pv,poi_uv,partition_date
0,49283687,9451,2105,665,2017-06-16
1,49283687,9451,2385,645,2017-06-19
2,49283687,9451,2495,665,2017-06-25
3,49283687,9451,2335,685,2017-06-07
4,49283687,9451,3185,1065,2017-06-06


In [27]:
df_poi_deal_pv_test = pd.read_csv('../raw_data/poi_deal_pv_test.txt', sep='\t')

In [28]:
df_poi_deal_pv_train.head(5)

,deal_id,poi_id,poi_pv,poi_uv,partition_date
0,49283687,9451,2105,665,2017-06-16
1,49283687,9451,2385,645,2017-06-19
2,49283687,9451,2495,665,2017-06-25
3,49283687,9451,2335,685,2017-06-07
4,49283687,9451,3185,1065,2017-06-06


## 用户推荐菜

poi_id : 商户ID

menu_name : ???

In [29]:
df_user_recommended_dish = pd.read_csv('../raw_data/user_recommended_dish.txt', sep='\t')

In [30]:
df_user_recommended_dish.head(5)

,poi_id,menu_name
0,6566309,3457987
1,5866252,3457987
2,40170526,3457987
3,5856510,3457987
4,5923860,3457987


## 商家推荐菜

menu_name : ???

poi_id : 商户ID

price : 菜品价格

rec_cnt : 推荐次数

In [31]:
df_merchant_recommended_dish = pd.read_csv('../raw_data/merchant_recommended_dish.txt', sep='\t')

In [32]:
df_merchant_recommended_dish.head(5)

,poi_id,price,rec_cnt,menu_name
0,2452017,14.0,10,3192575
1,164580933,22.0,10,93355
2,163991384,NaN,10,1392436
3,40242461,48.0,10,431080
4,40242461,46.0,10,1269049


## 菜品搭配

deal_id : 项目id

menu_name : 菜品名称
 
dishes_price : 菜品单价

dishes_group_name : 菜品集，荤菜，肉类，素菜，烤鱼，人气王之类

dish_tag : 菜的标签

 
整理后的菜品名

建议只使用菜品分类：主食、酒水饮品，汤羹粥，小食，菜品和果蔬生鲜，冷热标签准确率不是很好，荤素准确率可以使用， 可烤，不可烤标签可以使用

In [33]:
df_dish_train = pd.read_csv('../raw_data/dish_train.txt', sep='\t')
df_dish_test = pd.read_csv('../raw_data/dish_test.txt', sep='\t')

In [34]:
df_dish_train.head(5)

,deal_id,dishes_price,dishes_group_name,menu_name,dish_tag
0,48231610,86,17880,3340928,12|9|11
1,47742012,666,115752,3854864,7
2,47613346,74,163405,2808488,12|9|11
3,46969875,34,43952,3511080,12|9|10
4,47730420,32,116204,1836316,12|1|2


In [35]:
df_dish_test.head(5)

,deal_id,dishes_price,dishes_group_name,menu_name,dish_tag
0,50146548,14,192299,4026492,NaN
1,49827248,26,61296,708116,NaN
2,50691051,26,81974,4931466,NaN
3,49862531,22,188483,4931466,NaN
4,49870955,50,175426,3708519,NaN


## 套餐信息

deal_id : 项目id

begin_date : 上线日期

market_price : 原价

price : 卖价

deal_max_num : 最大用餐人数

deal_min_num : 最小用餐人数

deal_avg_num : 评价用餐人数

time_available(17:00~21:00,11:00~14:00) : 使用时间

day_unavailable(2018-01-31,2018-02-23) : 不可用时间

weekday_unavailable (6,7) : 不可用时间

poi_count : 套餐可用门店数

In [36]:
df_deal_train = pd.read_csv('../raw_data/deal_train.txt', sep='\t')
df_deal_test = pd.read_csv('../raw_data/deal_test.txt', sep='\t')

In [37]:
df_deal_train.head(5)

,deal_id,market_price,price,deal_max_num,deal_min_num,deal_avg_num,time_available,day_unavailable,weekday_unavailable,begin_date,poi_count
0,46015424,28.0,21.8,1.0,1.0,1.0,NaN,NaN,NaN,2016-12-07 00:00:00.000,1
1,46019208,2456.0,2170.0,12.0,10.0,11.0,09:00~次日02:00,NaN,NaN,2016-12-07 00:00:00.000,1
2,46025832,206.0,126.0,1.0,1.0,1.0,NaN,NaN,NaN,2016-12-06 00:00:00.000,1
3,46027033,192.0,148.0,2.0,2.0,2.0,9:00~21:00,NaN,NaN,2016-12-08 00:00:00.000,1
4,46029387,328.0,282.0,2.0,2.0,2.0,"11:00~14:00,17:00~21:00",NaN,NaN,2016-12-07 00:00:00.000,1


In [38]:
df_deal_test.head(5)

,deal_id,market_price,price,deal_max_num,deal_min_num,deal_avg_num,time_available,day_unavailable,weekday_unavailable,begin_date,poi_count
0,49586251,28.0,21.6,1.0,1.0,1.0,10:30~21:00,NaN,NaN,2017-08-05,1
1,49586266,66.0,49.2,1.0,1.0,1.0,10:00~21:00,NaN,NaN,2017-08-05,1
2,49586416,166.0,69.8,2.0,1.0,2.0,10:00~22:00,NaN,NaN,2017-08-19,1
3,49586551,70.0,60.0,1.0,1.0,1.0,10:30~20:30,NaN,NaN,2017-08-06,1
4,49586641,70.0,49.6,1.0,1.0,1.0,9:00~22:00,NaN,NaN,2017-08-09,1


## 套餐销量

deal_id : 项目id

poi_id : 门店id

sales : 项目销量

In [39]:
df_deal_sales_train = pd.read_csv('../raw_data/deal_sales_train.txt', sep='\t')

In [40]:
df_deal_sales_train.head(5)

,deal_id,poi_id,sales
0,25157093,4111280,46
1,27536586,5548875,16
2,27559562,815585,12
3,27559562,815586,12
4,27851864,4496366,14


## 套餐与门店关系

In [41]:
df_deal_poi_test = pd.read_csv('../raw_data/deal_poi_test.txt', sep='\t')

In [42]:
df_deal_poi_test.head(5)

,deal_id,poi_id
0,50162920,4665
1,50258846,5814
2,49575632,6889
3,50470857,6890
4,50398782,9247
